In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import azureml.core
from azureml.core import Workspace, Run

In [ ]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

In [ ]:
# Create script folder 
import os 
script_folder = os.path.join(os.getcwd(), 'script') 
os.makedirs(script_folder, exist_ok=True) 

In [ ]:
%%writefile script/train.py

import os
import time
import azureml
import argparse
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras import utils, losses
from keras.models import Sequential
from azureml.core import Workspace, Run
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten


os.environ["TF_CPP_MIN_LOG_LEVEL"]= "2"
os.environ['KERAS_BACKEND'] = 'tensorflow'

#Fashion MNIST Dataset CNN model development: https://github.com/zalandoresearch/fashion-mnist
from keras.datasets import fashion_mnist


parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
args = parser.parse_args()

# declare variables for model training
num_classes = 10
batch_size = 128
epochs = 10
img_rows,img_cols = 28,28

#load training and testing data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, sep = '\n')

# Define the labels
fashion_mnist_labels = ["Top",          
                        "Trouser",      
                        "Jumper",        
                        "Dress",         
                        "Coat",         
                        "Sandal",       
                        "Shirt",        
                        "Trainer",       
                        "Bag",          
                        "Ankle boot"]   

#data pre-processing
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test,  num_classes)

#formatting issues for depth of image (greyscale = 1) with different kernels (tensorflow, cntk, etc)
if K.image_data_format()== 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0],1,img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols,1)
    x_test = x_test.reshape(x_test.shape[0],img_rows, img_cols,1)
    input_shape = (img_rows, img_cols,1)
    
    
# model for image classification
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), padding = 'same', activation = 'relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, kernel_size=(3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

# start an Azure ML run
run = Run.get_context()

print('Train a deep learning model')
model.compile(loss=losses.categorical_crossentropy, optimizer="Adam", metrics=['accuracy'])
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

# evaluate the model performance on test data
print('Predict the test set')
result = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss: ', result[0])
print('Test Accuracy: ', result[1])

# calculate accuracy on the prediction
print('Accuracy is', result[1])
run.log('accuracy', np.float(result[1]))

os.makedirs('outputs/model', exist_ok=True) 

# save trained model 
model.save('outputs/model/model.h5') 


In [ ]:
# Get attached Arc-enabled kubernetes compute
arc_compute = ws.compute_targets["arc-ml-demo"] 

In [ ]:
from azureml.core import Experiment
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import ScriptRunConfig

# Register environment to re-use later
env = Environment('test')
conda_dep = packages = CondaDependencies.create(conda_packages=['pip', 'tensorflow', 'keras', 'scikit-learn', 'h5py'],
                                    pip_packages=['azureml-defaults'])
env.python.conda_dependencies = conda_dep
env.register(workspace = ws)

# reference the data
datastore = ws.get_default_datastore()
data_ref = datastore.path('./data').as_mount()

src = ScriptRunConfig(source_directory=script_folder,
                      script='train.py',
                      arguments=['--data-folder', str(data_ref)],
                      compute_target=arc_compute,
                      environment=env)
                      
# Run the experiment
src.run_config.data_references = {data_ref.data_reference_name: data_ref.to_config()}
run = Experiment(workspace=ws, name='fashion-mnist').submit(src)
run.wait_for_completion(show_output=True)


In [ ]:
print(run.get_metrics())

In [ ]:
# get files in experiment record
print(run.get_file_names())

In [ ]:
run.download_file(name="./outputs/model/model.h5", output_file_path="./outputs/model/model.h5")

In [ ]:
# register model 
model = run.register_model(model_name='fashion-mnist', model_path='./outputs/model/model.h5')
print(model.name, model.id, model.version, sep = '\t')